<a href="https://colab.research.google.com/github/rubaahmedkhan/Gemini-Experiments/blob/main/Text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[GEMNI-DOCUMENT]( https://ai.google.dev/gemini-api/docs/text-generation?lang=python.)

# **Google Gen AI SDK (experimental)**
The new Google Gen AI SDK provides a unified interface to Gemini 2.0 through both the Gemini Developer API and the Gemini API on Vertex AI. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

The Gen AI SDK also supports the Gemini 1.5 models.

The new SDK is available in Python and Go, with Java and JavaScript coming soon.

You can start using the SDK as shown below.

Install the new SDK: pip install google-genai

**Setup  
Install SDK**

In [ ]:
!pip install -U -q google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.3 MB/s eta 0:00:00


# **Set up your API key**

In [ ]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

**Import**

Import all the necessary modules.



In [ ]:
from google import genai

# **Initialize SDK client**
The client will pick up your API key from the environment variable. To use the live API you need to set the client version to v1alpha.

In [ ]:

client = genai.Client()


In [ ]:

MODEL: str = "gemini-2.0-flash-exp"


# **Text to Text**

In [16]:
response = client.models.generate_content(
    model=MODEL, contents='what is life? explain in one line'
)
print(response.text)

Life is a complex and dynamic process characterized by organization, metabolism, growth, adaptation, response to stimuli, and reproduction.



**Generate text from image input**
[GOOGLE-CODE](https://ai.google.dev/gemini-api/docs/text-generation?lang=python#generate-text-from-text-and-image)

In [13]:
!curl -o image.jpg "https://i.sstatic.net/0Jl54.png"


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  107k  100  107k    0     0   384k      0 --:--:-- --:--:-- --:--:--  386k


In [15]:
from PIL import Image
from google import genai

client = genai.Client()

image = Image.open("/content/image.jpg")
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[image, "what are the massege in this picture"])
print(response.text)

The message in the picture is the opening lines of Charles Dickens' novel "A Tale of Two Cities". It describes a time of great contrasts and contradictions.



# **Generate a text stream**
By default, the model returns a response after completing the entire text generation process. You can achieve faster interactions by not waiting for the entire result, and instead use streaming to handle partial results.

In [19]:
from google import genai

client = genai.Client()

response = client.models.generate_content_stream(
    model="gemini-2.0-flash",
    contents=["Explain machine learning "])
for chunk in response:
    print(chunk.text, end="")

Okay, let's break down machine learning in a clear and understandable way.

**What is Machine Learning?**

At its core, machine learning (ML) is a type of artificial intelligence (AI) that allows computer systems to learn from data without being explicitly programmed.  Instead of being told *exactly* how to solve a problem, the system is given data, and it figures out the rules and patterns on its own.  Think of it like teaching a child to recognize a cat. You don't give them a list of rules ("cats have pointed ears, whiskers, and a tail...") You show them lots of pictures of cats, and eventually, they learn to identify cats, even ones they've never seen before.

**Key Idea: Learning from Data**

The central principle of machine learning is that with enough data, a computer can learn to:

*   **Identify patterns:** Recognize trends, relationships, and structures within the data.
*   **Make predictions:**  Forecasting future outcomes based on past data.
*   **Make decisions:**  Automati

# **Create a chat conversation**
The Gemini SDK lets you collect multiple rounds of questions and responses, allowing users to step incrementally toward answers or get help with multipart problems. This SDK feature provides an interface to keep track of conversations history, but behind the scenes uses the same generateContent method to create the response.

The following code example shows a basic chat implementation:

In [20]:
from google import genai

client = genai.Client()

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)
for message in chat._curated_history:
    print(f'role - ', message.role, end=": ")
    print(message.parts[0].text)

That's wonderful! Dogs bring so much joy and companionship. Do you want to tell me anything else about them? Like their names, breeds, or what you enjoy doing with them?

Since you have 2 dogs, and each dog has 4 paws, there are 8 paws in your house.

role -  user: I have 2 dogs in my house.
role -  model: That's wonderful! Dogs bring so much joy and companionship. Do you want to tell me anything else about them? Like their names, breeds, or what you enjoy doing with them?

role -  user: How many paws are in my house?
role -  model: Since you have 2 dogs, and each dog has 4 paws, there are 8 paws in your house.



**You can also use streaming with chat, as shown in the following example:**

In [22]:
from google import genai

client = genai.Client()

chat = client.chats.create(model="gemini-2.0-flash")
response = chat.send_message_stream("I have 2 dogs in my house.")
for chunk in response:
    print(chunk.text, end="")
response = chat.send_message_stream("How many paws are in my house?")
for chunk in response:
    print(chunk.text, end="")
for message in chat._curated_history:
    print(f'role - ', message.role, end=": ")
    print(message.parts[0].text)

Okay! Two dogs can certainly add a lot of fun (and maybe some fur!) to a household. Do you want to tell me anything else about your dogs? Like their names, breeds, or what you enjoy doing with them?
Since you have two dogs, and each dog has four paws, there are 2 * 4 = 8 paws belonging to your dogs. And assuming you live there and have two feet (which equal two paws), that means there are 8 + 2 = 10 paws in your house.
role -  user: I have 2 dogs in my house.
role -  model: Okay
role -  model: !
role -  model:  Two dogs can certainly add a lot of fun (and maybe some fur!) to a household
role -  model: . Do you want to tell me anything else about your dogs? Like their names, breeds
role -  model: , or what you enjoy doing with them?

role -  user: How many paws are in my house?
role -  model: Since
role -  model:  you have two dogs, and each dog has four paws, there are 2 *
role -  model:  4 = 8 paws belonging to your dogs. And assuming you live there
role -  model:  and have two feet (

# **Configure text generation**
Every prompt you send to the model includes parameters that control how the model generates responses. You can use GenerationConfig to configure these parameters. If you don't configure the parameters, the model uses default options, which can vary by model.

The following example shows how to configure several of the available options.

In [23]:
from google import genai
from google.genai import types

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=["Explain how AI works in one line"],
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0.1
    )
)
print(response.text)

AI works by training algorithms on data to recognize patterns and make predictions or decisions.



#**Add system instructions**
System instructions let you steer the behavior of a model based on your specific needs and use cases.

By giving the model system instructions, you provide the model additional context to understand the task, generate more customized responses, and adhere to specific guidelines over the full user interaction with the model. You can also specify product-level behavior by setting system instructions, separate from prompts provided by end users.

You can set system instructions when you initialize your model:

In [27]:
sys_instruct="You are a maths teacher. Your name is Ruba."
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=sys_instruct),
    contents=["what is linear algebra"]
)
response.text

"Hello! I'm Ruba, your maths teacher.\n\nLinear algebra is a branch of mathematics that deals with **vectors, vector spaces, and linear transformations** between those spaces. Think of it as the maths behind many things you see and use every day, from computer graphics to data analysis!\n\nHere's a breakdown of the key concepts:\n\n*   **Vectors:** A vector is an object that has both magnitude (length) and direction. Think of it like an arrow. In linear algebra, we often represent vectors as lists of numbers. For example, (2, 3) represents a vector in a 2-dimensional space.\n\n*   **Vector Spaces:** A vector space is a collection of vectors that satisfies certain rules, allowing us to perform operations like addition and scalar multiplication (multiplying a vector by a number). The familiar 2D and 3D spaces we learn about in geometry are examples of vector spaces, but vector spaces can have any number of dimensions.\n\n*   **Linear Transformations:** A linear transformation is a functi